Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
!pip install numerapi
!pip install vecstack;

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=6be762a04947a8a0fa7ddd59368e84b36d37682b9f838b9be8d9853a41aa03f7
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [ ]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [ ]:
seed = 1
rand = np.random.seed(seed)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-08-28 18:09:19,409 INFO numerapi.utils: starting download
./numerai_dataset_279.zip: 100%|█████████▉| 419M/419M [00:11<00:00, 40.6MB/s]2021-08-28 18:09:31,233 INFO numerapi.base_api: unzipping file...
./numerai_dataset_279.zip: 419MB [00:29, 40.6MB/s]                           

In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

./numerai_dataset_279.zip: 419MB [00:32, 12.8MB/s]

# Loading data...


# All Loaded...
CPU times: user 1min 16s, sys: 36.4 s, total: 1min 52s
Wall time: 1min 53s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [ ]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [ ]:
%%time

model = GradientBoostingRegressor(random_state=rand)
model.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()



CPU times: user 33min 39s, sys: 1.43 s, total: 33min 41s
Wall time: 33min 30s


In [ ]:

y_pred_model = model.predict(tournament_data[feature_names])
gc.collect()

tournament_data[PREDICTION_NAME] = y_pred_model

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.488127
n000920ed083903f    0.490204
n0038e640522c4a6    0.522209
n004ac94a87dc54b    0.503401
n0052fe97ea0c05f    0.501119
                      ...   
nffd1cda0c61ecdb    0.504454
nffd711b0e6007e3    0.505101
nffd8a4d389df16a    0.498968
nffdec2abd3d4c1e    0.506943
nfff8751e4f86103    0.497827
Name: prediction, Length: 1800934, dtype: float64

In [ ]:
def neutralize(df, columns, by, proportion=1.0):
  
    scores = df[columns]
    exposures = df[by].values
    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))
    gc.collect()
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    gc.collect()
    return scores / scores.std()


In [ ]:
%%time
tournament_data[PREDICTION_NAME] = neutralize(tournament_data, PREDICTION_NAME, feature_names)

2021-08-28 18:47:59,494 INFO numexpr.utils: NumExpr defaulting to 4 threads.


CPU times: user 5min 19s, sys: 1min 9s, total: 6min 29s
Wall time: 2min 13s


In [ ]:
def minmax_norm(df):
    return (df - df.min()) / (df.max() - df.min())

In [ ]:
tournament_data[PREDICTION_NAME] = minmax_norm(tournament_data[PREDICTION_NAME])

In [ ]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.545544
n000920ed083903f    0.414119
n0038e640522c4a6    0.564908
n004ac94a87dc54b    0.697689
n0052fe97ea0c05f    0.572014
                      ...   
nffd1cda0c61ecdb    0.508803
nffd711b0e6007e3    0.468174
nffd8a4d389df16a    0.478302
nffdec2abd3d4c1e    0.477360
nfff8751e4f86103    0.466961
Name: prediction, Length: 1800934, dtype: float64

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [25]:
tournament_data[PREDICTION_NAME].to_csv('BeeChain_2.csv')

!cp BeeChain_2.csv "drive/My Drive/BeeChain Foundation/"